# MLP for Regression and Classificaiton using both PyTorch and Tensorflow

### Step 0: Set Up the Environment
Ensure you have PySpark, PyTorch, and other necessary libraries installed.

In [1]:
# ! pip install pyspark torch torchvision

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from keras import layers, Input
from keras.utils import to_categorical

## Part 1: MLP for Regression – Predicting California Housing Prices



### Step 1: Load and Prepare the Data
* Load the California housing dataset into a Pandas DataFrame.
* Convert it into a Spark DataFrame for distributed processing.
* Assemble feature vectors for PySpark.

In [3]:
# Load and Normalize California Housing Data
housing = fetch_california_housing()
df_housing = pd.DataFrame(housing.data, columns=housing.feature_names)
df_housing['target'] = housing.target  # Apply log transformation

X = df_housing[housing.feature_names].values
y = df_housing['target'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Step 2: [PyTorch] Convert Spark Data to PyTorch Tensors

In [4]:
# Convert NumPy arrays to PyTorch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


### Step 3: [PyTorch] Build the MLP Model for Regression

In [5]:
# Define MLP architecture to match TensorFlow
class MLPRegressor(nn.Module):
    def __init__(self, input_size):
        super(MLPRegressor, self).__init__()
        # First layer with BatchNorm
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        # Second layer with ReLU and then Dropout (matching TensorFlow placement)
        self.layer2 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3)  # Moved dropout after ReLU to match TensorFlow
        )
        # Output layer
        self.layer3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

# Create model
pt_model_regression = MLPRegressor(X_train.shape[1])

# Use Adam optimizer with default settings to match TensorFlow
# TensorFlow default Adam: lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-7
optimizer = optim.Adam(
    pt_model_regression.parameters(),
    lr=0.001,  # Default TensorFlow learning rate
    betas=(0.9, 0.999),  # Default betas
    eps=1e-8  # PyTorch default epsilon (slightly different from TF)
)


### Step 4: [PyTorch] Train the Regression Model

In [6]:
# Loss function
criterion = nn.MSELoss()

# Create DataLoader for mini-batch training
train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Match TF batch size=32

# Training loop with mini-batches (50 epochs to match TensorFlow)
num_epochs = 50
for epoch in range(num_epochs):
    running_loss = 0.0

    # Mini-batch training
    for batch_x, batch_y in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = pt_model_regression(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print statistics (similar to TensorFlow verbose=1)
    if epoch % 10 == 0 or epoch == num_epochs-1:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


Epoch [1/50], Loss: 0.9534
Epoch [11/50], Loss: 0.4374
Epoch [21/50], Loss: 0.3995
Epoch [31/50], Loss: 0.3900
Epoch [41/50], Loss: 0.3798
Epoch [50/50], Loss: 0.3785


### Step 5: [PyTorch] Evaluate the Regression Model

In [7]:
# Evaluate model
pt_model_regression.eval()
with torch.no_grad():
    predictions = pt_model_regression(X_test_t)
    mse_loss = criterion(predictions, y_test_t)
    mae_loss = nn.L1Loss()(predictions, y_test_t)  # Calculate MAE to match TensorFlow metrics

print(f'Test Mean Squared Error: {mse_loss.item():.4f}')
print(f'Test Mean Absolute Error: {mae_loss.item():.4f}')


Test Mean Squared Error: 0.4117
Test Mean Absolute Error: 0.4383


### Step 6: [PyTorch] Make Predictions with the Regression Model

In [8]:
# Make predictions with scaled input (matching TensorFlow's approach)
sample_input_np = np.array([[8.0, 41.0, 6.0, 1.0, 950.0, 4.0, 37.0, -122.0]])
sample_input_scaled = scaler.transform(sample_input_np)  # Apply scaling
sample_input_t = torch.tensor(sample_input_scaled, dtype=torch.float32)

pt_model_regression.eval()
with torch.no_grad():
    pt_prediction = pt_model_regression(sample_input_t).item()

print(f'Predicted House Price: {pt_prediction:.2f}')


Predicted House Price: 3.95


### Step 7: [TensorFlow] Build the MLP Model for Regression

In [9]:
# Define MLP architecture
tf_model_regression = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dense(32, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1)
])

# Compile the model
tf_model_regression.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Display model summary
tf_model_regression.summary()


c:\Users\craig\anaconda3\envs\ml-env\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,817 (11.00 KB)

 Non-trainable params: 128 (512.00 B)

### Step 8: [TensorFlow] Train the Regression Model

In [10]:
# Train the model
tf_history_regression = tf_model_regression.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    verbose=1
)


Epoch 1/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.5204 - mae: 0.9177 - val_loss: 0.5018 - val_mae: 0.4880
Epoch 2/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - loss: 0.6610 - mae: 0.5957 - val_loss: 0.5604 - val_mae: 0.5350
Epoch 3/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - loss: 0.5509 - mae: 0.5363 - val_loss: 0.4975 - val_mae: 0.5005
Epoch 4/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - loss: 0.4809 - mae: 0.4991 - val_loss: 0.5079 - val_mae: 0.5049
Epoch 5/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 0.4715 - mae: 0.4915 - val_loss: 0.4414 - val_mae: 0.4625
Epoch 6/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - loss: 0.4334 - mae: 0.4706 - val_loss: 0.4019 - val_mae: 0.4260
Epoch 7/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step - loss: 0.4436 - mae: 0.4713 - val_loss: 0.4123 - val_mae: 0.4401
Epoch 8/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - loss: 0.4315 - mae: 0.4655 - val_loss: 0.3333 - val_mae: 0.3950
Epoch 9/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 0s

### Step 9: [TensorFlow] Evaluate the Regression Model

In [11]:
# Evaluate the model on test data
loss, mae = tf_model_regression.evaluate(X_test, y_test)
print(f"Test Mean Absolute Error: {mae:.4f}")


129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - loss: 0.3207 - mae: 0.4148
Test Mean Absolute Error: 0.4166


### Step 10:[TensorFlow] Make Predictions with the Regression Model

In [12]:
# Sample input for prediction
sample_input = np.array([[8.0, 41.0, 6.0, 1.0, 950.0, 4.0, 37.0, -122.0]])
sample_input = scaler.transform(sample_input)  # Apply same scaling

# Make prediction
predicted_price = tf_model_regression.predict(sample_input)[0, 0]
print(f"Predicted House Price: {predicted_price:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
Predicted House Price: 2.83


## Part 2: MLP for Classification – Predicting Iris Flower Species

### Step 11: Load and Prepare the Data
* Load the Iris dataset into a Pandas DataFrame.
* Convert it into a Spark DataFrame.
* Encode categorical labels and assemble features.

In [13]:
# Load dataset
iris = load_iris()
df_iris = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df_iris['label'] = iris.target

# Convert to NumPy arrays
X = df_iris[iris.feature_names].values
y = df_iris['label'].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Step 12: (PyTorch) Convert Spark Data to PyTorch Tensors

In [14]:
# Convert to PyTorch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)  # Long tensor for classification
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)


### Step 13: (PyTorch) Build the MLP Model for Classification

In [15]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize model
input_size = X_train_t.shape[1]
hidden_size = 16
output_size = len(iris.target_names)

pt_model_classification = MLPClassifier(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pt_model_classification.parameters(), lr=0.01)




### Step 14: (PyTorch) Train the Classification Model

In [16]:
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = pt_model_classification(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [0/100], Loss: 1.0945
Epoch [10/100], Loss: 0.9142
Epoch [20/100], Loss: 0.8036
Epoch [30/100], Loss: 0.7375
Epoch [40/100], Loss: 0.6997
Epoch [50/100], Loss: 0.6680
Epoch [60/100], Loss: 0.6362
Epoch [70/100], Loss: 0.6146
Epoch [80/100], Loss: 0.6019
Epoch [90/100], Loss: 0.5944


### Step 15: (PyTorch) Evaluate the Classification Model

In [17]:
with torch.no_grad():
    predictions = pt_model_classification(X_test_t)
    predicted_labels = torch.argmax(predictions, axis=1)
    accuracy = (predicted_labels == y_test_t).sum().item() / y_test_t.size(0)

print(f'Classification Accuracy: {accuracy:.4f}')

Classification Accuracy: 0.9667


### Step 16: (PyTorch) Make Predictions with the Classification Model

In [18]:
sample_input = torch.tensor([[6.7, 3.1, 4.9, 1.5]])  # Example from dataset
with torch.no_grad():
    prediction = pt_model_classification(sample_input)
    predicted_class = torch.argmax(prediction, axis=1).item()

print(f'Predicted Class: {iris.target_names[predicted_class]}')


Predicted Class: virginica


### Step 17: (TensorFlow) Build the MLP Model for Classification

In [19]:
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)

# Create a new model (to avoid sharing weights)
tf_model_classification = keras.Sequential([
    Input(shape=(X_train.shape[1],)),  # Define input layer
    layers.Dense(16, activation="relu"),
    layers.Dense(8, activation="relu"),
    layers.Dense(3, activation="softmax")
])

# Compile with categorical crossentropy
tf_model_classification.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

tf_model_classification.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 243 (972.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 18: (TensorFlow) Train the Classification Model

In [20]:
# Train the model
history_classification = tf_model_classification.fit(
    X_train, y_train_one_hot,
    validation_data=(X_test, y_test_one_hot),
    epochs=100,
    batch_size=16,
    verbose=1
)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.1082 - loss: 1.0891 - val_accuracy: 0.3000 - val_loss: 1.0683
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2602 - loss: 1.0589 - val_accuracy: 0.3667 - val_loss: 1.0337
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2629 - loss: 1.0357 - val_accuracy: 0.4000 - val_loss: 1.0026
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3025 - loss: 1.0174 - val_accuracy: 0.6333 - val_loss: 0.9730
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5630 - loss: 0.9738 - val_accuracy: 0.6333 - val_loss: 0.9419
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5770 - loss: 0.9465 - val_accuracy: 0.7000 - val_loss: 0.9075
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6610 - loss: 0.9234 - val_accuracy: 0.8000 - val_loss: 0.8712
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7757 - loss: 0.8969 - val_accuracy: 0.9000 - val_loss: 0.8346

### Step 19: (TensorFlow) Evaluate the Classification Model

In [21]:
# Evaluate the model on test data
# Use one-hot encoded labels for evaluation
loss, accuracy = tf_model_classification.evaluate(X_test, y_test_one_hot)
print(f"Test Accuracy: {accuracy:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.0593
Test Accuracy: 1.0000


### Step 20: Make Predictions with the Classification Model

In [22]:
# Sample input for prediction
sample_input = np.array([[6.7, 3.1, 4.9, 1.5]])  # Example from Iris dataset

# Make prediction
tf_model_classification_prediction = tf_model_classification.predict(sample_input)
tf_predicted_class = np.argmax(tf_model_classification_prediction, axis=1)[0]

# Output predicted class
print(f"Predicted Class: {iris.target_names[predicted_class]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Class: virginica
